In all versions 3.x the narrative gets cleaned as before and products are cleaned once times by randomly removing etries with the three most common values for product. This run uses 10 different principle components. 

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_first_cut.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 6


__Initializing the bag of words__

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=10)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(292470, 10)


In [4]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.44366173 -3.22163153  0.01532147 ...  0.04225807 -0.67049439
  -1.16483872]
 [ 3.81940418 -1.11038977  0.3335882  ... -0.67660939  0.52543506
  -0.76600207]
 [11.86264199  4.43885137  4.55404876 ... -1.57353676 -0.66009409
   2.84084153]
 ...
 [ 7.78327094  3.84664355 -3.26286347 ...  0.36599344  8.3838297
  -2.80009118]
 [ 1.06129822  0.45520749 -0.09378672 ...  0.37944113  0.0375239
   0.14162473]
 [ 0.93802157  0.23180713  0.38582534 ...  0.30408381 -0.03706027
  -0.35475707]]
                0         1         2         3         4         5         6  \
0        4.443662 -3.221632  0.015321  0.403247  0.058375  0.573759 -0.708096   
1        3.819404 -1.110390  0.333588 -1.003640 -1.508024  1.678368  1.521393   
2       11.862642  4.438851  4.554049  0.525245  3.169759 -2.711624 -3.335482   
3        2.077610  1.348541  1.202235 -0.666619 -2.075840  0.831785  1.339177   
4        5.176562 -4.094481 -1.813897 -4.857587  1.599608  2.178552 -0.289792   
...           ...       

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(292470, 10)
(292470, 161)
(292470, 171)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(219352, 171)
(73118, 171)
(219352,)
(73118,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3682
                                                 Checking or savings account       0.99      1.00      1.00      4576
                                                               Consumer Loan       0.94      0.78      0.85      2827
                                                                 Credit card       1.00      1.00      1.00      4734
                                                 Credit card or prepaid card       0.96      0.97      0.96      7776
                                                            Credit reporting       1.00      1.00      1.00      7955
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.91      0.95     11684
                                                       